In [48]:
import pandas as pd
import csv
import pymorphy3
from pymorphy3 import MorphAnalyzer
import time
from tqdm import tqdm


import numpy as np
import re
import string
import nltk
from collections import defaultdict
from sklearn import metrics
from time import time
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

from nltk.tokenize import word_tokenize
import requests

import sklearn
sklearn.set_config(transform_output='pandas')

Датасет от Валерии

In [6]:
data_1 = pd.read_csv('csv_file.csv')
data_1.head(6)

,page_url,image_url,author,title,annotation
0,https://www.biblio-globus.ru//product/10957168,https://static1.bgshop.ru/imagehandler.ashx?fi...,Маринина А.,Ричард Третий и Генрих Восьмой глазами Шекспира,Расследование жизни и деяний двух самых опороч...
1,https://www.biblio-globus.ru//product/10997075,https://static1.bgshop.ru/imagehandler.ashx?fi...,Михалкова Е.И.,Знак истинного пути,Сыщики Макар Илюшин и Сергей Бабкин расследуют...
2,https://www.biblio-globus.ru//product/10941892,https://static1.bgshop.ru/imagehandler.ashx?fi...,Маринина А.,Генрих Шестой глазами Шекспира,Генрих Шестой – самая трагическая фигура на ан...
3,https://www.biblio-globus.ru//product/10978116,https://static1.bgshop.ru/imagehandler.ashx?fi...,Маринина А.,"Иоанн Безземельный, Эдуард Третий и Ричард Вто...",Истории трех знаменитых королей под одной обло...
4,https://www.biblio-globus.ru//product/10845007,https://static1.bgshop.ru/extraimagehandler.as...,Маринина А.,Шпаргалка для ленивых любителей истории. Корол...,Первая книга в необычном для Александры Марини...
5,https://www.biblio-globus.ru//product/10928065,https://static1.bgshop.ru/imagehandler.ashx?fi...,Елена Ивановна Михалкова,"Мертвый кролик, живой кролик","Самый громкий провал Илюшина: преступление, ко..."


In [11]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2149 entries, 0 to 2148
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   page_url    2149 non-null   object
 1   image_url   2149 non-null   object
 2   author      2143 non-null   object
 3   title       2149 non-null   object
 4   annotation  2146 non-null   object
dtypes: object(5)
memory usage: 84.1+ KB


In [24]:
filtered_data_1 = data_1.dropna(subset=['author'])

In [26]:
filtered_data_1 = filtered_data_1.dropna(subset=['annotation'])
filtered_data_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2140 entries, 0 to 2148
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   page_url    2140 non-null   object
 1   image_url   2140 non-null   object
 2   author      2140 non-null   object
 3   title       2140 non-null   object
 4   annotation  2140 non-null   object
dtypes: object(5)
memory usage: 100.3+ KB


Датасет от Савра

In [8]:
data_2 = pd.read_csv('csv_file_Savr.csv', on_bad_lines='skip')

In [9]:
data_2.head(5)

,page_url,image_url,author,title,annotation
0,https://www.biblio-globus.ru//product/10983286,https://static1.bgshop.ru/imagehandler.ashx?fi...,Лю Цысинь,Темный лес (Воспоминания о прошлом Земли #2. Б...,Трисолярианский кризис продолжается. У землян ...
1,https://www.biblio-globus.ru//product/10864732,https://static1.bgshop.ru/imagehandler.ashx?fi...,Чэнь Д.,"Вайолет, созданная из шипов (#1)",Вайолет — предсказательница. Но никто не знае...
2,https://www.biblio-globus.ru//product/10988948,https://static1.bgshop.ru/imagehandler.ashx?fi...,Лю Цысинь,Задача трех тел (Воспоминания о прошлом Земли ...,"В те времена, когда Китай переживал последстви..."
3,https://www.biblio-globus.ru//product/10922727,https://static1.bgshop.ru/imagehandler.ashx?fi...,Парди К.,Рассвет костяной волшебницы (#2),Священный долг каждой костяной волшебницы – пе...
4,https://www.biblio-globus.ru//product/10864123,https://static1.bgshop.ru/imagehandler.ashx?fi...,Кунц Д.,Холодный огонь,"Их свел случай, странное происшествие, которое..."


In [12]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3531 entries, 0 to 3530
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   page_url    3531 non-null   object
 1   image_url   1970 non-null   object
 2   author      1375 non-null   object
 3   title       920 non-null    object
 4   annotation  581 non-null    object
dtypes: object(5)
memory usage: 138.1+ KB


In [27]:
filtered_data_2 = data_2.dropna(subset=['annotation'])
filtered_data_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 581 entries, 0 to 3529
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   page_url    581 non-null    object
 1   image_url   581 non-null    object
 2   author      579 non-null    object
 3   title       581 non-null    object
 4   annotation  581 non-null    object
dtypes: object(5)
memory usage: 27.2+ KB


In [28]:
filtered_data_2 = filtered_data_2.dropna(subset=['author'])
filtered_data_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 579 entries, 0 to 3529
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   page_url    579 non-null    object
 1   image_url   579 non-null    object
 2   author      579 non-null    object
 3   title       579 non-null    object
 4   annotation  579 non-null    object
dtypes: object(5)
memory usage: 27.1+ KB


In [39]:
data_1_2 = pd.concat([filtered_data_1, filtered_data_2], ignore_index=True) # объединенный с библиоглобуса

In [33]:
data_1_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2719 entries, 0 to 2718
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   page_url    2719 non-null   object
 1   image_url   2719 non-null   object
 2   author      2719 non-null   object
 3   title       2719 non-null   object
 4   annotation  2719 non-null   object
dtypes: object(5)
memory usage: 106.3+ KB


Датасет от Сауле (с loveread)

In [29]:
data_3 = pd.read_csv('books_v7_500.csv')

In [31]:
data_3.tail(5)

,page_url,image_url,author,title,annotation
495,http://loveread.ec/view_global.php?id=10138,http://loveread.ec/img/photo_books/10138.jpg,Нил Шустерман,Книга Громила,Когда Бронте начинает встречаться с Брюстером ...
496,http://loveread.ec/view_global.php?id=10150,http://loveread.ec/img/photo_books/10150.jpg,Шеррилин Кеньон,Книга Дьявол может плакать [= Син и Катра ],"Месть. Некоторые говорят, что это яд, просачив..."
497,http://loveread.ec/view_global.php?id=10162,http://loveread.ec/img/photo_books/10162.jpg,Бекка Фицпатрик,Книга Тишина,Ссоры между Патчем и Норой закончились. Они ра...
498,http://loveread.ec/view_global.php?id=10166,http://loveread.ec/img/photo_books/10166.jpg,Элизабет Гоудж,Книга Маленькая белая лошадка в серебряном све...,…Лунная Долина полнится воспоминаниями о Лунно...
499,http://loveread.ec/view_global.php?id=10206,http://loveread.ec/img/photo_books/10206.jpg,Кэтрин Коултер,Книга Дикая звезда,"С той секунды, как очаровательная Байрони впер..."


In [34]:
data_3['title'] = data_3['title'].str.replace(r'^Книга\s+', '', regex=True)
data_3.head(5)

,page_url,image_url,author,title,annotation
0,http://loveread.ec/view_global.php?id=41,http://loveread.ec/img/photo_books/41.jpg,Владимир Набоков,Приглашение на казнь,"Это - собрание самых, наверное, странных произ..."
1,http://loveread.ec/view_global.php?id=1166,http://loveread.ec/img/photo_books/1166.jpg,Агата Кристи,Третья девушка,Молодая девушка по имени Норма Рестарик обраща...
2,http://loveread.ec/view_global.php?id=1187,http://loveread.ec/img/photo_books/1187.jpg,Роджер Желязны,Принеси мне голову Прекрасного принца,Несоврешеннолетний демон Аззи вообразил себя н...
3,http://loveread.ec/view_global.php?id=1289,http://loveread.ec/img/photo_books/1289.jpg,Гарри Гаррисон,Стальную крысу - в президенты !,Великолепный Джим ди Гриз - знаменитый межзвез...
4,http://loveread.ec/view_global.php?id=1352,http://loveread.ec/img/photo_books/1352.jpg,Юз Алешковский,Рука,"Роман Юза Алешковского «Рука» (1977, опубл. 19..."


In [35]:
data_main = pd.concat([data_1_2, data_3], ignore_index=True)

In [36]:
data_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3219 entries, 0 to 3218
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   page_url    3219 non-null   object
 1   image_url   3219 non-null   object
 2   author      3219 non-null   object
 3   title       3219 non-null   object
 4   annotation  3219 non-null   object
dtypes: object(5)
memory usage: 125.9+ KB


In [40]:
data_4 = pd.read_csv('books_1000.csv')
data_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726 entries, 0 to 725
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   page_url    726 non-null    object
 1   image_url   726 non-null    object
 2   author      726 non-null    object
 3   title       726 non-null    object
 4   annotation  726 non-null    object
dtypes: object(5)
memory usage: 28.5+ KB


In [41]:
data_main = pd.concat([data_main, data_4], ignore_index=True)

In [43]:
data_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3945 entries, 0 to 3944
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   page_url    3945 non-null   object
 1   image_url   3945 non-null   object
 2   author      3945 non-null   object
 3   title       3945 non-null   object
 4   annotation  3945 non-null   object
dtypes: object(5)
memory usage: 154.2+ KB


In [42]:
duplicates = data_main[data_main['title'].duplicated(keep=False)]
duplicates

,page_url,image_url,author,title,annotation
7,https://www.biblio-globus.ru//product/10974400,https://static1.bgshop.ru/imagehandler.ashx?fi...,Свечин Н.,Ледяной ветер Суоми,"Немудрено, что кассир крупного банка, уроженец..."
15,https://www.biblio-globus.ru//product/10974399,https://static1.bgshop.ru/imagehandler.ashx?fi...,Свечин Н.,Ледяной ветер Суоми,"Немудрено, что кассир крупного банка, уроженец..."
31,https://www.biblio-globus.ru//product/10893792,https://static1.bgshop.ru/imagehandler.ashx?fi...,Елена Ивановна Михалкова,Перо бумажной птицы,Пропала девушка. Ее любящая семья нанимает час...
33,https://www.biblio-globus.ru//product/10871804,https://static1.bgshop.ru/imagehandler.ashx?fi...,Бочарова Т.А.,Восстание Феникса,Всю жизнь Регина Сергеевна проработала учитель...
46,https://www.biblio-globus.ru//product/10989481,https://static1.bgshop.ru/imagehandler.ashx?fi...,Корецкий Д.А.,Возвращение не гарантируется,"Они познакомились на пустынном ночном шоссе, г..."
...,...,...,...,...,...
3423,http://loveread.ec/view_global.php?id=12245,http://loveread.ec/img/photo_books/12245.jpg,Лилия Белая,Книга Мечта,"Что ты будешь делать, когда, попадав в другой ..."
3655,http://loveread.ec/view_global.php?id=14473,http://loveread.ec/img/photo_books/14473.jpg,Эмиль Золя,Книга Мечта,"Стояла суровая зима 1860 года, Уаза замерзла, ..."
3800,http://loveread.ec/view_global.php?id=15496,http://loveread.ec/img/photo_books/15496.jpg,Франц Кафка,Книга Превращение,"В настоящий том вошли роман Кафки «Замок», при..."
3849,http://loveread.ec/view_global.php?id=16065,http://loveread.ec/img/photo_books/16065.jpg,Келли Армстронг,Книга Пробуждение,"Если бы мы встретились несколько недель назад,..."


In [44]:
data_main_no_duplicates = data_main.drop_duplicates(subset='title', keep='first')

In [45]:
data_main_no_duplicates.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3691 entries, 0 to 3944
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   page_url    3691 non-null   object
 1   image_url   3691 non-null   object
 2   author      3691 non-null   object
 3   title       3691 non-null   object
 4   annotation  3691 non-null   object
dtypes: object(5)
memory usage: 173.0+ KB


Preproccesing

In [51]:
url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ru/master/stopwords-ru.txt"
stop_words_russian = requests.get(url).text.splitlines()

In [52]:
def clean(text):
    text = re.sub(r'([А-Я])', r' \1', text) # делаем пробел между верхним и нижним регистром(123/473)
    text = text.lower() # нижний регистр (194)
    text = re.sub(r'[\w\.-]+\.ru', " ", text) # удаляем сайты (4)
    text = re.sub(r'http\S+', " ", text) # удаляем ссылки (10/1076)
    text = re.sub(r'@\w+',' ',text) # удаляем упоминания пользователей (1937/1923)
    text = re.sub(r'#\w+', ' ', text) # удаляем хэштеги (10/77/1923)
    text = re.sub(r'\d+', ' ', text) # удаляем числа()
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Удаление английских слов
    text = ' '.join(re.findall(r'\b[а-яА-ЯёЁ]+\b', text))
    # Удаление стоп-слов
    tokens = word_tokenize(text)
    text = ' '.join([word for word in tokens if word.lower() not in stop_words_russian])
    # text = re.sub(r'<.*?>',' ', text) #
    text = re.sub(r'['u'\U0001F600-\U0001F64F'
                  u'\U0001F300-\U0001F5FF'
                  u'\U0001F680-\U0001F6FF'
                  u'\U0001F1E0-\U0001F1FF'
                  u'\U00002500-\U00002BEF'
                  u'\U00002702-\U000027B0'
                  u'\U000024C2-\U0001F251'
                  u'\U0001f926-\U0001f937'
                  u'\U00010000-\U0010ffff'
                    ']+', '', text, flags=re.UNICODE) # удаляем эмоджи(4/1707)
    return text

In [54]:
data_main_no_duplicates['cleaned_text'] = data_main_no_duplicates['annotation'].apply(clean)
data_cleaned = data_main_no_duplicates['cleaned_text']
data_cleaned

/tmp/ipykernel_3152/2295399223.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_main_no_duplicates['cleaned_text'] = data_main_no_duplicates['annotation'].apply(clean)


0       расследование жизни деяний самых опороченных в...
1       сыщики макар илюшин сергей бабкин расследуют п...
2       генрих самая трагическая фигура английском пре...
3       истории трех знаменитых королей обложкой посво...
4       первая необычном александры марининой жанре по...
                              ...                        
3940    ирма повести бизнесплан аспирантки помышляла б...
3941    основная идея романа давление обстоятельств жи...
3942    небольшой горный пансионат игл нест туристичес...
3943    пола привыкла разговоры любви духовном физичес...
3944    последняя пристройка крепости мак леннан завер...
Name: cleaned_text, Length: 3691, dtype: object

In [55]:
morph = pymorphy3.MorphAnalyzer()

In [58]:
def lemmatize_batch(texts, batch_size=1000):
    """Лемматизация текста по батчам для ускорения."""
    lemmatized_texts = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        lemmatized_batch = [' '.join([morph.parse(word)[0].normal_form for word in text.split()]) for text in batch]
        lemmatized_texts.extend(lemmatized_batch)
    return lemmatized_texts


In [59]:
data_main_no_duplicates['lemmatized_text'] = lemmatize_batch(data_main_no_duplicates['cleaned_text'].tolist())

100%|█████████████████████████████████████████████| 4/4 [00:20<00:00,  5.15s/it]
/tmp/ipykernel_3152/3826416347.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_main_no_duplicates['lemmatized_text'] = lemmatize_batch(data_main_no_duplicates['cleaned_text'].tolist())


In [61]:
data_main_no_duplicates['lemmatized_text']

0       расследование жизнь деяние самый опорочить воз...
1       сыщик макар илюшин сергей бабкин расследовать ...
2       генрих самый трагический фигура английский пре...
3       история три знаменитый король обложка посвоему...
4       первый необычный александра маринин жанр попул...
                              ...                        
3940    ирма повесть бизнесплан аспирантка помышлять б...
3941    основный идея роман давление обстоятельство жи...
3942    небольшой горный пансионат игла несть туристич...
3943    пол привыкнуть разговор любовь духовный физиче...
3944    последний пристройка крепость мак леннана заве...
Name: lemmatized_text, Length: 3691, dtype: object

In [ ]:
data_main_no_duplicates['lemmatized_text'].to_csv('lemmatized_data')

Убрать из названия знаки препинания